In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import tree , datasets
import os

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# data source path
data_path = r'./data'
final_data_path = r'./final_data'



In [22]:
# data read and rename columns 
birth_col = ['cow_number','confinement date','Date of dried milk','calf number1','calf number2','cow weight','login date'
            ,'Calculated tire','tire','difficulty level of confinement','calf size','calf gender','farm number']

report_col = ['ID','year','month','farm number','cow_number','father_cow_number','mother_cow_number','birthday','tire'
              ,'lactation date','milk support','last confinement date','sampling date','age of month','inspection date'
             ,'final breeding date','final breeding semen','breeding times','before confinement date','first breeding date'
             ,'first breeding semen']

birth_df = pd.read_csv(os.path.join(data_path , 'birth.csv') , names = birth_col,  skiprows=1)
report_df = pd.read_csv(os.path.join(data_path , 'report.csv') , names = report_col , skiprows=1)

# new data get for 11/25 date
report_df = pd.concat([report_df, pd.read_csv(os.path.join(final_data_path , 'report.csv') , names = report_col , skiprows=1)])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
print('birth table : ')
print ("Dataset Shape:: ", birth_df.shape)
birth_df.head()

birth table : 
Dataset Shape::  (3761, 13)


,cow_number,confinement date,Date of dried milk,calf number1,calf number2,cow weight,login date,Calculated tire,tire,difficulty level of confinement,calf size,calf gender,farm number
0,87121677,2013/1/4 00:00,NaN,NaN,NaN,NaN,2013/1/25 00:00,True,2,1.0,NaN,NaN,B
1,96126940,2013/1/2 00:00,NaN,NaN,NaN,522.0,2013/1/25 00:00,True,2,2.0,NaN,NaN,B
2,95181708,2013/1/14 00:00,2014/1/3 00:00,2188113,NaN,670.0,2013/1/25 00:00,True,5,1.0,NaN,NaN,C
3,97127665,2013/1/9 00:00,2014/3/12 00:00,NaN,NaN,NaN,2013/1/25 00:00,True,3,2.0,NaN,NaN,B
4,97182634,2013/1/9 00:00,2014/3/25 00:00,NaN,NaN,630.0,2013/1/25 00:00,True,4,1.0,NaN,NaN,C


In [24]:
print('report table : ')
print ("Dataset Shape:: ", report_df.shape)
report_df.head()

report table : 
Dataset Shape::  (39946, 21)


,ID,year,month,farm number,cow_number,father_cow_number,mother_cow_number,birthday,tire,lactation date,...,last confinement date,sampling date,age of month,inspection date,final breeding date,final breeding semen,breeding times,before confinement date,first breeding date,first breeding semen
0,1,2013,3,A,52612,507H7173,96040545,2011/1/1 00:00,1,78.0,...,2013/1/1 00:00,2013/3/20 00:00,26,2013/3/21 00:00,2015/5/5 00:00,507H11621,1,NaN,2013/3/5 00:00,501H8784
1,2,2013,4,A,52612,507H7173,96040545,2011/1/1 00:00,1,107.0,...,2013/1/1 00:00,2013/4/18 00:00,27,2013/4/19 00:00,2015/5/5 00:00,507H11621,1,NaN,2013/3/5 00:00,501H8784
2,3,2013,5,A,52612,507H7173,96040545,2011/1/1 00:00,1,140.0,...,2013/1/1 00:00,2013/5/21 00:00,28,2013/5/22 00:00,2015/5/5 00:00,507H11621,2,NaN,2013/3/5 00:00,501H8784
3,4,2013,6,A,52612,507H7173,96040545,2011/1/1 00:00,1,175.0,...,2013/1/1 00:00,2013/6/25 00:00,29,2013/6/26 00:00,2015/5/5 00:00,507H11621,2,NaN,2013/3/5 00:00,501H8784
4,5,2013,7,A,52612,507H7173,96040545,2011/1/1 00:00,1,210.0,...,2013/1/1 00:00,2013/7/30 00:00,30,2013/7/31 00:00,2015/5/5 00:00,507H11621,2,NaN,2013/3/5 00:00,501H8784


In [25]:
# change string to int 

def replace_string_int(x , items):
    for i,j in items:
        if x == j :
            return str(i) 

def change_col(col_list,report_df):
    for col in col_list:
        farm_number_list = list(set(report_df[col]))
        turple_fram_number = [i for i in enumerate(farm_number_list)]
        #print(turple_fram_number)
        report_df[col] = report_df[col].apply(lambda x : replace_string_int(x , turple_fram_number))
        
    return report_df
    
change_col_list = ['farm number','final breeding semen','first breeding semen','father_cow_number','mother_cow_number']
report_df = change_col(change_col_list , report_df)




In [26]:
def counts_with_months(x):
    return (int(x['year']) - 2013)*12 + int(x['month'])
# generator time series 
report_df['months'] = report_df.apply(lambda x : counts_with_months(x) , 1)


In [99]:
temp_df = report_df.drop(['year','month','birthday','lactation date','last confinement date','sampling date','inspection date'
                   ,'final breeding date','before confinement date','first breeding date'],axis=1)
temp_df = temp_df.dropna()
x = temp_df.drop('milk support',axis=1)
y = temp_df['milk support']

# add feature to 12 
x['empty'] = 0

In [28]:
import numpy as np
import torch,  os
import pandas as pd
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from torch import nn, optim
import torch.nn.functional as F


In [29]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels, transforms=None):
        self.X = images
        self.y = labels
        self.transforms = transforms
        
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        data = self.X.iloc[i]
        data = np.array(data)
        
        if self.transforms:
            data = self.transforms(data)
            
            # Also return label if we have it
            if self.y is not None:
                return (data, self.y[i])
            else:
                return data    

In [30]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

LEARNING_RATE = 1e-4
BATCH_SIZE = 128
NUM_EPOCHS = 15
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


INPUT_DIM = x.shape[1]
EMBEDDING_DIM = 64
HIDDEN_DIM = 128
OUTPUT_DIM = 1



In [100]:
# split data and reset index data  
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size = 0.1, random_state = 10)
x_train.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [101]:
# test data 
test_data = report_df[report_df['milk support'].isna()]
test_data = test_data.drop('milk support',axis=1)

In [102]:
import torch.nn as nn
import torch.optim as optim
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Model, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lstm = nn.LSTMCell(self.input_size, self.hidden_size)
        self.linear = nn.Linear(self.hidden_size, self.output_size)
    def forward(self, input, future=0, y=None):
        outputs = []
        # reset the state of LSTM
        # the state is kept till the end of the sequence
        h_t = torch.zeros(input.size(0), self.hidden_size, dtype=torch.float32)
        c_t = torch.zeros(input.size(0), self.hidden_size, dtype=torch.float32)
        print(input.size())
        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm(input_t, (h_t, c_t))
            output = self.linear(h_t)
            outputs += [output]
        for i in range(future):
            if y is not None and random.random() > 0.5:
                output = y[:, [i]]  # teacher forcing
            h_t, c_t = self.lstm(output, (h_t, c_t))
            output = self.linear(h_t)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

In [104]:
x_train = torch.from_numpy(np.array(x_train).astype('float'))
target = torch.from_numpy(np.array(y_train))
x_test = torch.from_numpy(np.array(x_test).astype('float'))
y_test = torch.from_numpy(np.array(y_test))

EMBEDDING_DIM = 64
HIDDEN_DIM = 3
OUTPUT_DIM = 1
INPUT_DIM=1


torch.manual_seed(RANDOM_SEED)
model = Model(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = model.to('cpu')
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [89]:
model

Model(
  (lstm): LSTMCell(1, 128)
  (linear): Linear(in_features=128, out_features=1, bias=True)
)

In [105]:

criterion = F.binary_cross_entropy_with_logits
#begin to train
for i in range(15):
    print('STEP: ', i)
    input_= x_train.unsqueeze(0).type(torch.float)
    optimizer.zero_grad()
    out = model(input_)
    loss = criterion(out, target)
    print('loss:', loss.item())
    loss.backward()
    optimizer.step()
    
    # begin to predict, no need to track gradient here
    with torch.no_grad():
        future = 1000
        pred = seq(x_test, future=future)
        loss = criterion(pred[:, :-future], y_test)
        print('test loss:', loss.item())
        #y = pred.detach().numpy()


STEP:  0
torch.Size([1, 26133, 12])


RuntimeError: size mismatch, m1: [1 x 12], m2: [1 x 12] at c:\programdata\miniconda3\conda-bld\pytorch_1533086652614\work\aten\src\th\generic/THTensorMath.cpp:2070

In [87]:
input_= x_train.unsqueeze(0).type(torch.float)

In [88]:
input_.size()

torch.Size([1, 26133, 11])

In [109]:
lstm = nn.LSTMCell(1,3)

In [110]:
by_one = input_.chunk(input_.size(1), dim=1)[0]

In [119]:
by_one_t = by_one.view(1,12,1)

In [122]:
lstm(by_one)

RuntimeError: size mismatch, m1: [1 x 12], m2: [1 x 12] at c:\programdata\miniconda3\conda-bld\pytorch_1533086652614\work\aten\src\th\generic/THTensorMath.cpp:2070